# Imports

In [3]:
import numpy as np
import pandas as pd
import nfl_data_py as nfl

# Importing NFL Data

In [4]:
years = range(1999, 2023 + 1)

# Loading Basic Data

Loading in our current version of the filter data - this takes forever to grab due to the wikipedia queries.

In [5]:
df_core = pd.read_pickle("../../FantasyData/data-frames/df_basic_ngs_1999_2023.pkl")

In [6]:
df_core.head()

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,avg_cushion,avg_separation,avg_intended_air_yards_receiving,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation
0,00-0000003,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00-0000003,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00-0000003,2000,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00-0000007,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00-0000007,2000,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Loading Snap Data

In [15]:
snap_years = range(2012, 2023 + 1)
df_snaps = nfl.import_snap_counts(snap_years)

In [35]:
df_snaps.head()

,game_id,pfr_game_id,season,game_type,week,player,pfr_player_id,position,team,opponent,offense_snaps,offense_pct,defense_snaps,defense_pct,st_snaps,st_pct
0,2012_01_ATL_KC,201209090kan,2012,REG,1,Branden Albert,AlbeBr20,T,KC,ATL,69.0,1.0,0.0,0.0,5.0,0.17
1,2012_01_ATL_KC,201209090kan,2012,REG,1,Rodney Hudson,HudsRo00,C,KC,ATL,69.0,1.0,0.0,0.0,5.0,0.17
2,2012_01_ATL_KC,201209090kan,2012,REG,1,Jon Asamoah,AsamJo20,G,KC,ATL,69.0,1.0,0.0,0.0,5.0,0.17
3,2012_01_ATL_KC,201209090kan,2012,REG,1,Ryan Lilja,LiljRy20,G,KC,ATL,69.0,1.0,0.0,0.0,5.0,0.17
4,2012_01_ATL_KC,201209090kan,2012,REG,1,Eric Winston,WinsEr20,T,KC,ATL,69.0,1.0,0.0,0.0,5.0,0.17


In [16]:
df_snaps.columns

Index(['game_id', 'pfr_game_id', 'season', 'game_type', 'week', 'player',
       'pfr_player_id', 'position', 'team', 'opponent', 'offense_snaps',
       'offense_pct', 'defense_snaps', 'defense_pct', 'st_snaps', 'st_pct'],
      dtype='object')

In [36]:
df_snap_agg = df_snaps \
    .copy() \
    .query("game_type == 'REG'") \
    .groupby(by=['season', 'pfr_player_id']) \
    .agg({"offense_snaps": "sum", "offense_pct": "mean"})

df_snap_agg.reset_index(inplace=True)

In [37]:
df_snap_agg.head()

,season,pfr_player_id,offense_snaps,offense_pct
0,2012,AbduIs00,0.0,0.000000
1,2012,AbraJo00,0.0,0.000000
2,2012,AchoSa00,0.0,0.000000
3,2012,AdamJo01,22.0,0.035556
4,2012,AdamKr00,94.0,0.460000


# Load Roster Data

In [29]:
df_roster = pd.read_pickle("./construction_dfs/df_roster_99_23.pkl")

In [30]:
df_roster.columns

Index(['season', 'team', 'position', 'depth_chart_position', 'jersey_number',
       'status', 'player_name', 'first_name', 'last_name', 'birth_date',
       'height', 'weight', 'college', 'player_id', 'espn_id', 'sportradar_id',
       'yahoo_id', 'rotowire_id', 'pff_id', 'pfr_id', 'fantasy_data_id',
       'sleeper_id', 'years_exp', 'headshot_url', 'esb_id', 'gsis_it_id',
       'smart_id', 'entry_year', 'rookie_year', 'draft_club', 'ngs_position',
       'week', 'game_type', 'status_description_abbr', 'football_name',
       'draft_number', 'age'],
      dtype='object')

# Merge Snaps by Ids

In [49]:
df_ids = df_roster.query("season >= 2012")[['season', 'pfr_id', 'player_id']]
df_ids.dropna(inplace=True)
df_ids.rename(columns={"pfr_id": "pfr_player_id"}, inplace=True)
df_ids.head()

,season,pfr_player_id,player_id
4,2012,BatcCh00,00-0000865
8,2012,DrivDo00,00-0004541
10,2012,GonzTo00,00-0006101
12,2012,HassMa00,00-0007091
17,2012,MannPe00,00-0010346


In [52]:
df_merge = df_snap_agg.merge(df_ids, on=["season", "pfr_player_id"], how="left")
df_merge.dropna(inplace=True)
df_merge.head()

,season,pfr_player_id,offense_snaps,offense_pct,player_id
2,2012,AchoSa00,0.0,0.000000,00-0028041
3,2012,AdamJo01,22.0,0.035556,00-0029271
23,2012,AlleAn01,53.0,0.043750,00-0028163
27,2012,AlleDw00,907.0,0.778750,00-0029689
35,2012,AluaTy99,0.0,0.000000,00-0027862


# Merge Snap Data with Core Data

In [55]:
df_core_merge = df_core.merge(df_merge, on=["season", "player_id"], how="left")

In [58]:
df_core_merge.query("season >= 2012")[["season", "player_name", "offense_snaps", "offense_pct"]]

,season,player_name,offense_snaps,offense_pct
97,2012,Charlie Batch,129.0,1.000000
602,2012,Donald Driver,150.0,0.169231
876,2012,Tony Gonzalez,966.0,0.913125
877,2013,Tony Gonzalez,1003.0,0.933125
1012,2012,Matt Hasselbeck,395.0,0.773750
...,...,...,...,...
13330,2023,Bryce Young,1067.0,0.996875
13331,2023,Will Levis,528.0,0.912222
13332,2023,C.J. Stroud,967.0,0.984000
13333,2023,Anthony Richardson,173.0,0.652500


# Save New Info

In [61]:
df_core_merge.to_pickle("../../FantasyData/data-frames/df_basic_ngs_snaps_1999_2023.pkl")